# ワークフロー実行のための準備

このノートブックでは、ワークフロー実行のための準備を行います。 
上から順番にセルを実行してください。

## 1. SSH秘密鍵を.ssh/id_rsaに配置する

プロトタイプでは、SSH秘密鍵をjupyterhubの.ssh/id_rsaに、公開鍵をginリポジトリに登録することでSSH接続するため、初めにSSH秘密鍵を.ssh/id_rsaに配置してください。  
※.sshフォルダは新規作成してください。

## 2. SSH接続で書き戻しを行うためにssh鍵に権限を与える（ssh鍵は.sshディレクトリの中にid_rsaとして保存する）

In [ ]:
%%sh
chmod 600 ~/.ssh/id_rsa

## 3. SSH鍵の利用確認をオフにする

In [ ]:
%%sh
echo "host *\n\tStrictHostKeyChecking no\n\tUserKnownHostsFile /dev/null" >> ~/.ssh/config

## 4. dataladを使用するため、このリポジトリをdatalad datasetとする

In [ ]:
%%sh
cd ~/
datalad create --force

## 5. ワークフロー実行結果をginリポジトリに反映させるため、ginリポジトリをリモートリポジトリに設定する

In [ ]:
%%sh
mkdir ./tmp
git remote show origin > ./tmp/remote.txt

In [ ]:
import json
json_open = open('/home/jovyan/param_files/param_for_dataladsiblings.json', 'r')
json_load = json.load(json_open)
http = json_load['http']
ssh = json_load['ssh']

In [ ]:
with open('./tmp/remote.txt', 'r') as f:
    lines = [s.strip() for s in f.readlines()]
    ssh_push_url = lines[2].replace(http, ssh) 

In [ ]:
import os

os.system("rm -r ./tmp")

In [ ]:
from datalad import api

try:
    api.siblings(action='add', name='gin', url=ssh_push_url)
except:
    pass

## 4. .gitconfigにユーザー名とメールアドレスを登録する

    1. 以下のセルでgitアカウントのユーザー名とメールアドレスを入力してください。

In [ ]:
name = input()

In [ ]:
email = input()

    2. 入力されたユーザー名とメールアドレスを.gitconfigに登録します。

In [ ]:
import os

os.system("cd ~/")
os.system("git config --global user.name " + name)
os.system("git config --global user.email " + email)

## 5. .local配下をgit管理しないように.gitignoreで設定する

In [ ]:
%%sh
echo ".local/*" >> ~/.gitignore

## 6. 実験用フォルダ構成を初期化する

In [ ]:
import os

experiment_folder = "/home/jovyan/experiments"
os.makedirs(experiment_folder)

## 7. 必要なファイルをPACKAGEテンプレートに複製する

In [ ]:
%%sh
cd ~/
mkdir WORKFLOW/PACKAGE/.ssh
cp .ssh/id_rsa WORKFLOW/PACKAGE/.ssh/id_rsa
cp .ssh/config WORKFLOW/PACKAGE/.ssh/config
cp .gitconfig WORKFLOW/PACKAGE/.gitconfig
cp .gitconfig WORKFLOW/PACKAGE/.gitconfig

## 7. 変更内容を書き戻す

In [ ]:
import papermill as pm

pm.execute_notebook(
    '../util/datalad_save_push.ipynb',
    '-',
    parameters = dict(SAVE_MESSAGE = 'ワークフロー実行準備')
)